In [ ]:
#install.packages(c("tidyverse", "reshape2", "survminer", "survival", "splines", "coxme", "multcomp"))
library(tidyverse) # loads dplyr, ggplot2, stringr etc
library(reshape2) # for melt and cast

# The following packages are for analyzing mortality data and producing Kaplan-Meier plots:
library(survminer)
library(survival)
library(splines)
library(coxme)
library(multcomp)

In [ ]:
data <- read.csv("1_insect_mortality.csv",
                 header = TRUE)

fit <- survfit(Surv(time, death) ~ treatment, data = data)
print(fit, show.rmean=TRUE)
plot(fit, conf.int=TRUE)
summary(fit)

In [ ]:
survdiff(Surv(time,death)~treatment, data=data)

comp<-pairwise_survdiff(Surv(time, death) ~ treatment,
                        data = data, p.adjust.method = "fdr") #log rank compares the difference until an event (death) occurs
comp

In [ ]:
#Model with randomization effect by experiment
set.seed(2)
s1 = survfit(Surv(time,death)~treatment, data=data)
print(s1, show.rmean=TRUE)
plot(s1, conf.int=TRUE)
summary(s1)

In [ ]:
#Statistics
#Model random effect
data$treatment <- as.factor(data$treatment)
model <- coxme(Surv(time, death) ~ treatment+(1|biological_replicate), data=data)
anova(model)

summary(glht(model, linfct=mcp(treatment="Tukey")), test = adjusted(type = "bonferroni"))

In [ ]:
ggsurvplot(s1, data = data, 
           main = "Survival Feeding (11 Days)",
           size = 1, ggtheme = theme_classic(), 
           linetype=c("solid", "dotted", "dotdash", "dashed"), 
           xlab="Days", ylab="% of survival", palette=c("#887bb9", "#a35da5", "#69b19a", "#8ec372"))

In [ ]:
# Midpoint and endpoint snapshot

tables <- ggsurvtable(s1, data = data, color = "strata",
  y.text = FALSE, break.time.by = 0.5)

tables

midpoint <- data.frame(Soil = c("C10", "C112", "S16", "S7"),
                       Dead = c(40, 65, 90, 65),
                       Alive = c(60, 35, 10, 35))
midpoint <- melt(midpoint, id = "Soil")
names(midpoint) <- c("Soil", "State", "Percent")
midpoint$State <- relevel(midpoint$State, 'Alive')

p <- ggplot(midpoint, aes(x = Soil, y = Percent, fill = State)) +
        geom_bar(position="stack", stat = "identity") + 
        theme_classic() 

p


endpoint <- data.frame(Soil = c("C10", "C112", "S16", "S7"),
                       Dead = c(63.2, 80, 100, 80),
                       Alive = c(36.8, 20, 0, 20))
endpoint <- melt(endpoint, id = "Soil")
names(endpoint) <- c("Soil", "State", "Percent")
endpoint$State <- relevel(endpoint$State, "Alive")

q <- ggplot(endpoint, aes(x = Soil, y = Percent, fill = State)) +
        geom_bar(position="stack", stat = "identity") +
        theme_classic()

q